In [32]:
from bs4 import BeautifulSoup
from time import sleep
import requests
import pandas as pd

In [33]:
# 変数urlにSUUMOホームページのURLを格納する。東京都／目黒区 15万円以上～20万円以下 2LDK／3LDKのURLを格納
url = 'https://suumo.jp/jj/chintai/ichiran/FR301FC001/?ar=030&bs=040&pc=30&smk=&po1=25&po2=99&shkr1=03&shkr2=03&shkr3=03&shkr4=03&sc=13110&ta=13&cb=15.0&ct=20.0&md=07&md=10&et=9999999&mb=0&mt=9999999&cn=9999999&fw2=&?page={}'

# target_urlへのアクセス結果を、変数rに格納
r = requests.get(target_url)

for i in range(1, 4):
    print('d_listの大きさ：', len(d_list))
    # 変数target_urlに、アクセス先のURLを格納する
    target_url = url.format(i)

    # print()してtarget_urlを確認する
    print(target_url)

d_listの大きさ： 58
https://suumo.jp/jj/chintai/ichiran/FR301FC001/?ar=030&bs=040&pc=30&smk=&po1=25&po2=99&shkr1=03&shkr2=03&shkr3=03&shkr4=03&sc=13110&ta=13&cb=15.0&ct=20.0&md=07&md=10&et=9999999&mb=0&mt=9999999&cn=9999999&fw2=&?page=1
d_listの大きさ： 58
https://suumo.jp/jj/chintai/ichiran/FR301FC001/?ar=030&bs=040&pc=30&smk=&po1=25&po2=99&shkr1=03&shkr2=03&shkr3=03&shkr4=03&sc=13110&ta=13&cb=15.0&ct=20.0&md=07&md=10&et=9999999&mb=0&mt=9999999&cn=9999999&fw2=&?page=2
d_listの大きさ： 58
https://suumo.jp/jj/chintai/ichiran/FR301FC001/?ar=030&bs=040&pc=30&smk=&po1=25&po2=99&shkr1=03&shkr2=03&shkr3=03&shkr4=03&sc=13110&ta=13&cb=15.0&ct=20.0&md=07&md=10&et=9999999&mb=0&mt=9999999&cn=9999999&fw2=&?page=3


In [34]:
# target_urlへのアクセス結果を、変数rに格納
r = requests.get(target_url)

sleep(1)

# 取得結果を解析してsoupに格納
soup = BeautifulSoup(r.text)

In [35]:
# すべての物件情報(20件)を取得する
contents = soup.find_all('div', class_='cassetteitem')

# 各物件情報をforループで取得する
for content in contents:
    # 物件情報と部屋情報を取得しておく
    detail = content.find('div', class_='cassetteitem_content')
    table = content.find('table', class_='cassetteitem_other')
    
    # 物件情報から必要な情報を取得する
    title = detail.find('div', class_='cassetteitem_content-title').text
    address = detail.find('li', class_='cassetteitem_detail-col1').text
    access = detail.find('li', class_='cassetteitem_detail-col2').text
    age = detail.find('li', class_='cassetteitem_detail-col3').text

    # 部屋情報のブロックから、各部屋情報を取得する
    tr_tags = table.find_all('tr', class_='js-cassette_link')
    
    # 各部屋情報をforループで取得する
    for tr_tag in tr_tags:        
        
        # 部屋情報の行から、欲しい情報を取得する
        floor, price, first_fee, capacity = tr_tag.find_all('td')[2:6]
        
        # さらに細かい情報を取得する
        fee, management_fee = price.find_all('li')
        deposit, gratuity = first_fee.find_all('li')
        madori, menseki = capacity.find_all('li')

        # 取得したすべての情報を辞書に格納する
        d = {
            '物件名': title,
            '住所': address,
            'アクセス': access,
            '築年数': age,
            '階数': floor.text,
            '賃料': fee.text,
            '管理費': management_fee.text,
            '敷金': deposit.text,
            '礼金': gratuity.text,
            '間取り': madori.text,
            '面積': menseki.text
        }
        
        # 取得した辞書をd_listに格納する
        d_list.append(d)

In [36]:
df = pd.DataFrame(d_list)

In [37]:
df

,物件名,住所,アクセス,築年数,階数,賃料,管理費,敷金,礼金,間取り,面積
0,エルファーロ大岡山II,東京都目黒区大岡山１,\n東急目黒線/大岡山駅 歩7分\n東急大井町線/緑が丘駅 歩14分\n東急東横線/都立大学...,\n新築\n4階建\n,\r\n\t\t\t\t\t\t\t\t\t\t\t3階,19.7万円,10000円,-,19.7万円,2LDK,45.02m2
1,東急田園都市線 池尻大橋駅 3階建 築18年,東京都目黒区青葉台３,\n東急田園都市線/池尻大橋駅 歩10分\n東京メトロ日比谷線/中目黒駅 歩15分\nＪＲ山...,\n築18年\n3階建\n,\r\n\t\t\t\t\t\t\t\t\t\t\t2階,18.1万円,4000円,9.05万円,27.15万円,2LDK,56.74m2
2,東急田園都市線 池尻大橋駅 3階建 築18年,東京都目黒区青葉台３,\n東急田園都市線/池尻大橋駅 歩10分\n東京メトロ日比谷線/中目黒駅 歩15分\nＪＲ山...,\n築18年\n3階建\n,\r\n\t\t\t\t\t\t\t\t\t\t\t2-3階,18.4万円,4000円,18.4万円,27.6万円,2LDK,56.74m2
3,東急東横線 都立大学駅 5階建 築34年,東京都目黒区中根１,\n東急東横線/都立大学駅 歩2分\n\n\n,\n築34年\n5階建\n,\r\n\t\t\t\t\t\t\t\t\t\t\t4階,19.5万円,5000円,19.5万円,-,2LDK,74.77m2
4,東急東横線 都立大学駅 5階建 築34年,東京都目黒区中根１,\n東急東横線/都立大学駅 歩2分\n\n\n,\n築34年\n5階建\n,\r\n\t\t\t\t\t\t\t\t\t\t\t4階,19.8万円,5000円,19.8万円,-,2LDK,74.77m2
...,...,...,...,...,...,...,...,...,...,...,...
111,ＣＯＴＥ２１,東京都目黒区下目黒２,\nＪＲ山手線/目黒駅 歩9分\n東急目黒線/不動前駅 歩9分\nＪＲ山手線/五反田駅 歩1...,\n築12年\n5階建\n,\r\n\t\t\t\t\t\t\t\t\t\t\t4階,17万円,5000円,17万円,17万円,2LDK,54.4m2
112,T’ｓ garden西小山,東京都目黒区目黒本町５,\n東急目黒線/西小山駅 歩5分\n東急目黒線/武蔵小山駅 歩9分\n東急目黒線/洗足駅 歩...,\n築11年\n4階建\n,\r\n\t\t\t\t\t\t\t\t\t\t\t3階,15.3万円,5000円,15.3万円,15.3万円,2LDK,40.12m2
113,ポンリーヴル柿の木坂,東京都目黒区柿の木坂１,\n東急東横線/都立大学駅 歩4分\n東急東横線/自由が丘駅 歩23分\n東急東横線/学芸大...,\n新築\n3階建\n,\r\n\t\t\t\t\t\t\t\t\t\t\t1-2階,17.5万円,7000円,17.5万円,17.5万円,2LDK,49.95m2
114,ポンリーヴル柿の木坂,東京都目黒区柿の木坂１,\n東急東横線/都立大学駅 歩4分\n東急東横線/自由が丘駅 歩23分\n東急東横線/学芸大...,\n新築\n3階建\n,\r\n\t\t\t\t\t\t\t\t\t\t\t2階,17.9万円,7000円,17.9万円,17.9万円,2LDK,43.44m2


In [39]:
# duplicated()で重複がしているかをTrueかFalseで返してくれます。
df.duplicated()

0      False
1      False
2      False
3      False
4      False
       ...  
111     True
112     True
113     True
114     True
115     True
Length: 116, dtype: bool

In [42]:
# drop_duplicates()で重複したデータを削除（できている？？）
# drop_duplicates(inplace=True)とすることで、処理したデータを出力だけでなく、出力したデータを元のdfに代入して変更してくれます。
df.drop_duplicates(inplace=True)
df

,物件名,住所,アクセス,築年数,階数,賃料,管理費,敷金,礼金,間取り,面積
0,エルファーロ大岡山II,東京都目黒区大岡山１,\n東急目黒線/大岡山駅 歩7分\n東急大井町線/緑が丘駅 歩14分\n東急東横線/都立大学...,\n新築\n4階建\n,\r\n\t\t\t\t\t\t\t\t\t\t\t3階,19.7万円,10000円,-,19.7万円,2LDK,45.02m2
1,東急田園都市線 池尻大橋駅 3階建 築18年,東京都目黒区青葉台３,\n東急田園都市線/池尻大橋駅 歩10分\n東京メトロ日比谷線/中目黒駅 歩15分\nＪＲ山...,\n築18年\n3階建\n,\r\n\t\t\t\t\t\t\t\t\t\t\t2階,18.1万円,4000円,9.05万円,27.15万円,2LDK,56.74m2
2,東急田園都市線 池尻大橋駅 3階建 築18年,東京都目黒区青葉台３,\n東急田園都市線/池尻大橋駅 歩10分\n東京メトロ日比谷線/中目黒駅 歩15分\nＪＲ山...,\n築18年\n3階建\n,\r\n\t\t\t\t\t\t\t\t\t\t\t2-3階,18.4万円,4000円,18.4万円,27.6万円,2LDK,56.74m2
3,東急東横線 都立大学駅 5階建 築34年,東京都目黒区中根１,\n東急東横線/都立大学駅 歩2分\n\n\n,\n築34年\n5階建\n,\r\n\t\t\t\t\t\t\t\t\t\t\t4階,19.5万円,5000円,19.5万円,-,2LDK,74.77m2
4,東急東横線 都立大学駅 5階建 築34年,東京都目黒区中根１,\n東急東横線/都立大学駅 歩2分\n\n\n,\n築34年\n5階建\n,\r\n\t\t\t\t\t\t\t\t\t\t\t4階,19.8万円,5000円,19.8万円,-,2LDK,74.77m2
5,東急東横線 都立大学駅 5階建 築34年,東京都目黒区中根１,\n東急東横線/都立大学駅 歩2分\n\n\n,\n築34年\n5階建\n,\r\n\t\t\t\t\t\t\t\t\t\t\t4階,19.8万円,10000円,19.8万円,-,2LDK,74.77m2
6,NK渋谷コータース,東京都目黒区大橋２,\n東急田園都市線/池尻大橋駅 歩6分\n京王井の頭線/神泉駅 歩9分\n京王井の頭線/駒場...,\n築56年\n11階建\n,\r\n\t\t\t\t\t\t\t\t\t\t\t2階,17.5万円,10000円,17.5万円,-,2LDK,52.77m2
7,NK渋谷コータース,東京都目黒区大橋２,\n東急田園都市線/池尻大橋駅 歩6分\n京王井の頭線/神泉駅 歩9分\n京王井の頭線/駒場...,\n築56年\n11階建\n,\r\n\t\t\t\t\t\t\t\t\t\t\t2階,17.5万円,15000円,17.5万円,-,2LDK,52.77m2
8,ヴィレッジ・イン・下目黒,東京都目黒区下目黒３,\n東急目黒線/不動前駅 歩7分\n東急目黒線/武蔵小山駅 歩13分\nＪＲ山手線/目黒駅 ...,\n築11年\n4階建\n,\r\n\t\t\t\t\t\t\t\t\t\t\t3階,17.2万円,6000円,17.2万円,17.2万円,2LDK,49.72m2
9,ヴィレッジ・イン・下目黒,東京都目黒区下目黒３,\n東急目黒線/不動前駅 歩7分\n東急目黒線/武蔵小山駅 歩13分\nＪＲ山手線/目黒駅 ...,\n築11年\n4階建\n,\r\n\t\t\t\t\t\t\t\t\t\t\t2階,17.4万円,6000円,17.4万円,17.4万円,2LDK,49.72m2


In [41]:
# dfをcsvに出力
df.to_csv('suumo.csv', index=None, encoding='utf-8-sig')